# RNN models
> Various RNNs and dynamical models.

In [ ]:
#| default_exp recurrent.models

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from wafer.basics import *

## Utils

In [ ]:
#| export
class Select(nn.Module):
    "Select from iterable."
    def __init__(self, idx=0):
        super().__init__()
        self.idx = idx
    def forward(self, x):
        return x[self.idx]

In [ ]:
#| export
def check_nonneg_weight(m):
    "Check the non-negative condition of weights."
    flag = True
    for n,p in m.named_parameters():
        if (p < 0).any(): print(f"{n}\n{p}\n{'-'*20}"); flag = False
    if flag: print('Passed.')

## Basic

In [ ]:
#| export
from wafer.init import default_init, lambda_init

In [ ]:
#| export
class SimpleRNN(nn.Module):
    """A RNN with its output mapped through a dense layer.

    Input:  x, shape (N, L, D_in) or (N, D_in)
    Output: outp, shape (N, L, D_out) or (N, D_out)
    """
    def __init__(self, ni, nh, no, num_layers: int=1,
                 actn: str='tanh',                     # nonlinearity, ['tanh', 'relu']
                 out_actn: Union[str, nn.Module]=None, # output activation
                 init: str='normal',                   # initialization method ['uniform', 'normal', 'irnn', 'np-rnn']
                 return_seq: bool=False,               # return the whole sequence
                ):
        super().__init__()
        self.recurrent = nn.RNN(ni, nh, num_layers=num_layers, batch_first=True, nonlinearity=actn)
        self.dense = nn.Linear(nh, no)
        self.out_actn = get_actn(out_actn)
        self.return_seq = return_seq
        # Initialize
        self._init(self.recurrent, nh, init)
        default_init(self.dense)

    @staticmethod
    @torch.no_grad()
    def _init(m, nh, init):
        for n,p in m.named_parameters():
            if 'bias' in n:
                nn.init.zeros_(p)
            if 'weight_ih' in n:
                if init == 'uniform':
                    nn.init.uniform_(p, a=-1/np.sqrt(nh), b=1/np.sqrt(nh))
                elif init == 'np-rnn':
                    nn.init.normal_(p, std=1/np.sqrt(nh))
                    p = p * (np.sqrt(2) * np.exp(1.2 / (max(nh, 6) - 2.4)))
                    getattr(m, n).copy_(p)
                else: nn.init.normal_(p, std=0.001)
            if 'weight_hh' in n:
                if init == 'uniform':
                    nn.init.uniform_(p, a=-1/np.sqrt(nh), b=1/np.sqrt(nh))
                elif init == 'irnn':
                    nn.init.eye_(p)
                elif init == 'np-rnn':
                    nn.init.normal_(p)
                    p = (p.T @ p) / nh
                    p = p / np.linalg.eigvals(p.detach().numpy()).max()
                    getattr(m, n).copy_(p)
                else: nn.init.normal_(p, std=1/np.sqrt(nh))
    
    def forward(self, x):
        output,_ = self.recurrent(x) # shape(N, L, nh) or (N, nh)
        if (output.ndim > 2) and (not self.return_seq): output = output[:,-1,:]
        return self.out_actn(self.dense(output))

In [ ]:
#| export
class LSTM(nn.Module):
    """Custom LSTM, allowing for different activations. Assuming `batch_first=True` and `bidirectional=False`.
    
    Inputs: x, h0_c0;
        x: shape (N, L, D_in) or (N, D_in).
        h0_c0: (h0, c0), Union[list, tuple], optional, default zeros, each of shape (num_layers, N, hidden_size).
    
    Outputs: output, (hn, cn);
        output: shape (N, L, hidden_size) or (N, hidden_size), outputs of the last layer for each token.
        hn: shape (num_layers, N, hidden_size), final hidden state.
        cn: shape (num_layers, N, hidden_size), final cell state.
    """
    def __init__(self, ni,
                 nh,
                 num_layers=1,
                 actn: Union[str, nn.Module]='tanh',
                 gate_actn: Union[str, nn.Module]='sigmoid',
                 dropout=0.0,
                 unit_forget_bias=True,
                 init_gain=1/np.sqrt(3),
                 recurrent_init_gain=1.,
                 return_seq=False
                ):
        super().__init__()
        self.nh,self.num_layers = nh,num_layers
        self.actn = get_actn(actn, nn.Tanh())
        self.gate_actn = get_actn(gate_actn, nn.Sigmoid())
        self.dropout = nn.Dropout(p=dropout)
        self.return_seq = return_seq

        ws = []
        for i in range(num_layers):
            ws.append(nn.ModuleDict({
                'ii': nn.Linear(ni if i == 0 else nh, nh),  # input-input weight
                'if': nn.Linear(ni if i == 0 else nh, nh),  # input-forget weight
                'io': nn.Linear(ni if i == 0 else nh, nh),  # input-output weight
                'ic': nn.Linear(ni if i == 0 else nh, nh),  # input-cell weight
                'hi': nn.Linear(nh, nh),                    # hidden-input weight
                'hf': nn.Linear(nh, nh),                    # hidden-forget weight
                'ho': nn.Linear(nh, nh),                    # hidden-output weight
                'hc': nn.Linear(nh, nh),                    # hidden-cell weight
            }))
        self.ws = nn.ModuleList(ws)
        
        # Initialize
        for n,p in self.named_parameters():
            # hidden/recurrent weight
            if '.h' in n:
                if 'bias' in n: nn.init.zeros_(p)
                else: nn.init.orthogonal_(p, recurrent_init_gain)
            # non-recurrent weight
            else:
                if 'bias' in n:
                    if '.if' in n and unit_forget_bias: nn.init.ones_(p)
                    else: nn.init.zeros_(p)
                else: nn.init.xavier_uniform_(p, init_gain) 
    
    def _forward_single(self, x, h0_c0: Union[list, tuple]=None):
        "Forward pass of a single token."
        if h0_c0 is None:
            h0 = torch.zeros(self.num_layers, x.shape[0], self.nh, device=x.device)
            c0 = torch.zeros(self.num_layers, x.shape[0], self.nh, device=x.device)
        else:
            h0,c0 = h0_c0
            assert (h0.shape[-1] == c0.shape[-1] == self.nh) and (h0.shape[1] == c0.shape[1] == x.shape[0])
        
        hs,cs = [],[]
        for i in range(self.num_layers):
            h,c = h0[i],c0[i]
            i_gate = self.ws[i]['ii'](x) + self.ws[i]['hi'](h)
            f_gate = self.ws[i]['if'](x) + self.ws[i]['hf'](h)
            o_gate = self.ws[i]['io'](x) + self.ws[i]['ho'](h)
            c_gate = self.ws[i]['ic'](x) + self.ws[i]['hc'](h)

            i_gate = self.gate_actn(i_gate)
            f_gate = self.gate_actn(f_gate)
            o_gate = self.gate_actn(o_gate)
            c_gate = self.actn(c_gate)

            c_new = (f_gate * c) + (i_gate * c_gate)
            h_new = o_gate * self.actn(c_new)
            cs.append(c_new)
            hs.append(h_new)
            x = self.dropout(h_new)

        hs,cs = torch.stack(hs, 0),torch.stack(cs, 0)
        return (hs, cs)

    def forward(self, x, h0_c0=None):
        # Input `x`, shape (N, L, D) or (N, D); `h0_c0` = (h0, c0), each shape (num_layers, N, nh).
        assert x.ndim in [2,3], f"Expect 2D or 3D input, but got {x.ndim}D."
        if x.ndim == 2: x = x.unsqueeze(1)

        hc = h0_c0
        if self.return_seq:
            hs = []
            for xi in x.permute(1,0,2):
                hc = self._forward_single(xi, hc)
                hs.append(hc[0])
            return torch.stack([h[-1] for h in hs], 1), hc
        else:
            for xi in x.permute(1,0,2):
                hc = self._forward_single(xi, hc)
            return hc[0][-1], hc

## Input convex

In [ ]:
#| export
class SimpleICRNN(nn.Module):
    """Simple input-convex RNN. `h = f(Wh_ + Ux + b); y = g(Vh + b)`, all weights are non-negative and
    activations are convex and non-decreasing. Weights are initialized as in [np-RNN](https://arxiv.org/abs/1511.03771).
    
    Inputs:  x, shape (N,L,D_in)
    Outputs: outputs, shape (N,L,D_out)
    """
    def __init__(self, ni: int,                        # input size
                 nh: int,                              # hidden size
                 no: int,                              # output size
                 actn: Union[str, nn.Module]='relu',   # hidden activation
                 out_actn: Union[str, nn.Module]=None, # output activation
                 expand_inp: bool=True,                # expand the input to [x, -x]
                 return_seq: bool=False                # return the whole seuqnece
                ):
        super().__init__()
        ni = ni * 2 if expand_inp else ni
        self.expand_inp,self.nh = expand_inp,nh
        self.return_seq = return_seq
        self.actn = get_actn(actn, nn.ReLU())
        self.out_actn = get_actn(out_actn)
        self.U = nn.Linear(ni, nh)
        self.V = nn.Linear(nh, no)
        self.W = nn.Linear(nh, nh)
        
        # Initialize
        with torch.no_grad():
            # recurrent weight
            nn.init.zeros_(self.W.bias)
            nn.init.normal_(self.W.weight, std=1)
            p = self.W.weight.clamp_min(0)
            p = (p.T @ p) / nh
            p = p / np.linalg.eigvals(p.detach().numpy()).max()
            self.W.weight.copy_(p)
            # input weight
            nn.init.zeros_(self.U.bias)
            nn.init.normal_(self.U.weight, std=1/np.sqrt(nh))
            p = self.U.weight * (np.sqrt(2) * np.exp(1.2 / (max(nh, 6) - 2.4)))
            self.U.weight.copy_(p)
            # output weight
            default_init(self.V)
        self.weight_constraint()

    def weight_constraint(self):
        "Apply nonnegative weight constriant."
        with torch.no_grad():
            for n,p in self.named_parameters():
                if 'weight' in n:
                    p.clamp_min_(0)

    def forward(self, x):
        # Input `x`, shape (N, L, D) or (N, D).
        assert x.ndim in [2,3], f"Expect 2D or 3D input, but got {x.ndim}D."
        if x.ndim == 2: x = x.unsqueeze(1)
        
        if self.expand_inp: x = torch.cat([x, -x], dim=-1)
        z = torch.zeros(x.shape[0], self.nh, device=x.device)
        if self.return_seq:
            ys = []
            for xi in x.permute(1,0,2):
                z = self.actn(self.U(xi) + self.W(z))
                ys.append(self.out_actn(self.V(z)))
            return torch.stack(ys, 1)
        else:
            for xi in x.permute(1,0,2):
                z = self.actn(self.U(xi) + self.W(z))
            return self.out_actn(self.V(z))

In [ ]:
#| export
class ICRNN(nn.Module):
    """Input-convex RNN. Refer to [Optimal Control Via Neural Networks: A Convex Approach](https://arxiv.org/abs/1805.11835).
    The recurrent weight matrix is initialized as `Identity * recurrent_init_gain`.
    
    Inputs:  x, shape (N,L,D_in)
    Outputs: outputs, shape (N,L,D_out)
    """
    def __init__(self, ni: int,                        # input size
                 nh: int,                              # hidden size
                 no: int,                              # output size
                 actn: Union[str, nn.Module]='relu',   # hidden activation
                 out_actn: Union[str, nn.Module]=None, # output activation
                 expand_inp: bool=True,                # expand the input to [x, -x]
                 init_gain: float=1.,                  # weight initialization gain
                 recurrent_init_gain: float=1.,        # recurrent weight initialization gain
                 return_seq: bool=False                # return the whole seuqnece
                ):
        super().__init__()
        ni = ni * 2 if expand_inp else ni
        self.expand_inp,self.nh = expand_inp,nh
        self.return_seq = return_seq
        self.actn = get_actn(actn, nn.ReLU())
        self.out_actn = get_actn(out_actn)
        self.U = nn.Linear(ni, nh)
        self.V = nn.Linear(nh, no)
        self.W = nn.Linear(nh, nh)
        self.D1 = nn.Linear(nh, no)
        self.D2 = nn.Linear(ni, nh)
        self.D3 = nn.Linear(ni, no)
        
        # Initialize
        lambda_init(self, lambda w,b: (nn.init.xavier_normal_(w, init_gain), nn.init.zeros_(b)))
        with torch.no_grad():
            self.W.weight.copy_(torch.eye(nh) * recurrent_init_gain)
        self.weight_constraint()

    def weight_constraint(self):
        "Apply nonnegative weight constriant."
        with torch.no_grad():
            for n,p in self.named_parameters():
                if 'weight' in n:
                    p.clamp_min_(0)

    def forward(self, x):
        # Input `x`, shape (N, L, D) or (N, D).
        assert x.ndim in [2,3], f"Expect 2D or 3D input, but got {x.ndim}D."
        if x.ndim == 2: x = x.unsqueeze(1)
        
        if self.expand_inp: x = torch.cat([x, -x], dim=-1)
        x = torch.cat([torch.zeros(x.shape[0],1,x.shape[-1], device=x.device), x], dim=-2)
        z_curr = torch.zeros(x.shape[0], self.nh, device=x.device)
        if self.return_seq:
            ys = []
            for xi,xj in zip(x[:,:-1,:].permute(1,0,2), x[:,1:,:].permute(1,0,2)):
                z_prev = z_curr
                z_curr = self.actn(self.U(xj) + self.W(z_prev) + self.D2(xi))
                ys.append(self.out_actn(self.V(z_curr) + self.D1(z_prev) + self.D3(xj)))
            return torch.stack(ys, 1)
        else:
            for xi,xj in zip(x[:,:-1,:].permute(1,0,2), x[:,1:,:].permute(1,0,2)):
                z_prev = z_curr
                z_curr = self.actn(self.U(xj) + self.W(z_prev) + self.D2(xi))
            return self.out_actn(self.V(z_curr) + self.D1(z_prev) + self.D3(xj))

In [ ]:
#| export
class WeightConstraintCB(Callback):
    "Weight constraint callback."
    order = 10
    def after_step(self): self.learner.model.weight_constraint()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()